In [84]:
# Dependencies
import requests
from config import api_key, CLIENT_ID, CLIENT_SECRET
import json
import pprint as pp
import pandas as pd

In [96]:
#Playlist for reference: https://open.spotify.com/playlist/20VO4SBwxES4tXbp1nvjk6?si=2kxhbktBSPua1xQd0xbq3w
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [97]:
# Build query URL

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

url = "https://api.spotify.com/v1/"

play_id = "20VO4SBwxES4tXbp1nvjk6?si=2kxhbktBSPua1xQd0xbq3w"

query_url = url+'playlists/'+play_id+'tracks'

In [98]:
# Get playlist data
json_req = requests.get(query_url, headers=headers)
response = json_req.json()

track_ids = []
artist_ids = []
track_names = []
artist_names = []
genres =[]

In [99]:
for item in response['tracks']['items']:
    track_names.append(item['track']['name'])
    track_ids.append(item['track']['id'])
    album_names.append(item['track']['album']['name'])
    album_ids.append(item['track']['album']['id'])
    genres.append(item['track']['artists'][0]['genres'])
df = pd.DataFrame({ 'Track Name':track_names, 'Album Name':album_names, 'Track ID':track_ids, 'Album ID': album_ids,  'Genres':genres})


KeyError: 'genres'

In [83]:
# Get Genre from each Album ID in separate JSON request
genres = []

#for album in album_ids:
#    query_url = url+'albums/'+album
#    json_req = requests.get(query_url, headers=headers)
#    response = json_req.json()
#    genres.append(response['genres'])
query_url = url+'artists/'+'2YZyLoL8N0Wb9xBt1NhZWg'
json_req = requests.get(query_url, headers=headers)
response = json_req.json()
pp.pprint(response)
#genres.append(response['album']['genres'])
#print(genres)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'},
 'followers': {'href': None, 'total': 24034094},
 'genres': ['conscious hip hop', 'hip hop', 'rap', 'west coast rap'],
 'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg',
 'id': '2YZyLoL8N0Wb9xBt1NhZWg',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5eb437b9e2a82505b3d93ff1022',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab67616100005174437b9e2a82505b3d93ff1022',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f178437b9e2a82505b3d93ff1022',
             'width': 160}],
 'name': 'Kendrick Lamar',
 'popularity': 89,
 'type': 'artist',
 'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}


In [ ]:
# Report temperature
print(f'The temperature in {city} is {data["main"]["temp"]} degrees Celsius.')